# 환경 설정

In [ ]:
#@title 0. 로컬(colab) 한글 환경 설정
# 그래프에 한글을 표기하기 위한 설정 (폰트 설치 + ...)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf  # 폰트 설치 후, 런타임 재시작 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (10.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121713 files and direc

# 데이터 수집


In [ ]:
#@title 1. API 데이터 수집 테스트
import requests

# url = 'https://apis.data.go.kr/B552584/EvCharger/getChargerInfo?serviceKey=여러분의_키&pageNo=1&numOfRows=10&zcode=26'
url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerInfo'
# url = 'http://apis.data.go.kr/B552584/EvCharger/getChargerStatus'

# svcKey는 decoding key (encoding 하지 않은 key라는 뜻에서 decoding key. 실제로 원래 키 -> 인코딩 -> 디코딩 은 아님)
svcKey = '여러분의_디코딩_키'
params = {'serviceKey' : svcKey, 'pageNo' : '1', 'numOfRows' : '10', 'zcode' : '26' }  # zcode; 26=부산, 11=서울

response = requests.get(url, params=params)
# response = requests.get(url)
print(response.content)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg><totalCount>28282</totalCount><pageNo>1</pageNo><numOfRows>10</numOfRows></header><body><items><item><statNm>\xeb\xb6\x80\xec\x82\xb0 \xec\x96\xb4\xeb\xa6\xb0\xec\x9d\xb4 \xeb\x8c\x80\xea\xb3\xb5\xec\x9b\x90</statNm><statId>ME181207</statId><chgerId>01</chgerId><chgerType>06</chgerType><addr>\xeb\xb6\x80\xec\x82\xb0\xea\xb4\x91\xec\x97\xad\xec\x8b\x9c \xeb\xb6\x80\xec\x82\xb0\xec\xa7\x84\xea\xb5\xac \xec\x84\xb1\xec\xa7\x80\xea\xb3\xa1\xeb\xa1\x9c 3</addr><addrDetail>null</addrDetail><location>null</location><lat>35.1831134</lat><lng>129.0459801</lng><useTime>24\xec\x8b\x9c\xea\xb0\x84 \xec\x9d\xb4\xec\x9a\xa9\xea\xb0\x80\xeb\x8a\xa5</useTime><busiId>ME</busiId><bnm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</bnm><busiNm>\xed\x99\x98\xea\xb2\xbd\xeb\xb6\x80</busiNm><busiCall>1661-9408</busiCall><stat>2</stat><statUpdDt>20251127195334</statUpdDt><lastTsdt

In [ ]:
#@title 2. 데이터 가져오기 위한 준비_1
# zcode에 대해, pageNo, numOfRows 를 지정해서 데이터를 가져오는 API 밖에 정의되어 있지 않네...
import xml.etree.ElementTree as ET

# Update params for the initial request
params['pageNo'] = '1'
params['numOfRows'] = '9999'
params['zcode'] = '26'

# Make the API request
response = requests.get(url, params=params)

# Parse the XML response
root = ET.fromstring(response.content)

# Extract totalCount
total_count = int(root.find('header').find('totalCount').text)

# Extract all item elements
all_items = root.find('body').find('items').findall('item')

print(f"Total count for zcode=26: {total_count}")
print(f"Number of items in the first page (up to 9999): {len(all_items)}")

Total count for zcode=26: 28282
Number of items in the first page (up to 9999): 9999


In [ ]:
#@title 3. 데이터 가져오기 위한 준비_2
# numOfRows를 최대치로 했는데 전부 못 가져왔으니, 나누어서 데이터를 가져와야겠네. 몇번에 나누어서 가져오면 좋을까?
import math

num_of_rows = int(params['numOfRows'])
total_pages = math.ceil(total_count / num_of_rows)

print(f"Total pages to retrieve all data for zcode=26: {total_pages}")

Total pages to retrieve all data for zcode=26: 3


In [ ]:
#@title 4. 데이터 가져오기 (메인)
import time

# Initialize a list to hold all collected items
all_items = []

for page_no in range(1, total_pages + 1):
    params['pageNo'] = str(page_no)
    response = requests.get(url, params=params)
    root = ET.fromstring(response.content)

    # Check if 'items' tag exists before trying to find 'item' elements
    items_element = root.find('body').find('items')
    if items_element is not None:
        all_items.extend(items_element.findall('item'))
    else:
        print(f"No items found on page {page_no}")

    # Introduce a small delay to avoid overwhelming the API server
    time.sleep(0.5)

print(f"Total items collected across all pages for zcode=26: {len(all_items)}")
print(all_items.head())
print(all_items[:5]) # Added: To show the first 5 elements of the list

Total items collected across all pages for zcode=26: 28282


AttributeError: 'list' object has no attribute 'head'

In [ ]:
#@title 5. 데이터 수집 확인
import pandas as pd

# Prepare a list to hold dictionaries of item data
data = []

# Iterate through each item element and extract relevant information
for item in all_items:
    item_data = {}
    for child in item:
        item_data[child.tag] = child.text
    data.append(item_data)

# Create a pandas DataFrame from the list of dictionaries
df = pd.DataFrame(data)

              statNm    statId chgerId chgerType                addr  \
0         부산 어린이 대공원  ME181207      01        06   부산광역시 부산진구 성지곡로 3   
1            부산시 교육청  ME181208      01        06   부산광역시 부산진구 화지로 12   
2  부산아시아드보조경기장VIP주차장  ME181210      01        06  부산광역시 연제구 거제동 1312   
3  부산아시아드보조경기장VIP주차장  ME181210      02        06  부산광역시 연제구 거제동 1312   
4  부산아시아드보조경기장VIP주차장  ME181210      11        04  부산광역시 연제구 거제동 1312   

  addrDetail location         lat          lng      useTime  ...  note  \
0       null     null  35.1831134  129.0459801    24시간 이용가능  ...  None   
1       null     null  35.1761301  129.0639265    24시간 이용가능  ...  None   
2       null     null  35.1883518  129.0555415  05:00~24:00  ...  None   
3       null     null  35.1883518  129.0555415  05:00~24:00  ...  None   
4       null     null  35.1883518  129.0555415  05:00~24:00  ...  None   

  limitYn limitDetail delYn delDetail trafficYn  year floorNum floorType maker  
0       N        None     N      None    

# 데이터 탐색

In [ ]:
#@title 1. 데이터 기본 정보 확인
# 데이터 샘플 확인
# print(df.head())
# print(df.tail())

# 전체 컬럼과 타입, 결측치 확인
# print(df.info())

# 행/열 개수
# print(df.shape)
print(df.count())

# 각 컬럼의 데이터 타입
# print(df.dtypes)

# 데이터 구조 파악
# print(df.columns)
# print(df.index)
# print(df.values)

# 특정 조건의 데이터 샘플 확인
# columns_object_dtype = df.columns[df.dtypes=='object']
# print(columns_object_dtype)
# for col in columns_object_dtype:
#     # print (sorted(df[col].unique()))
#     unique_values = sorted(df[col].unique())
#     print(f'{col}: {len(unique_values)}개')
#     print(unique_values, '\n')

# colab의 cell 안에서 모든 컬럼을 다 표시하기 위한 설정
# pd.set_option('display.max_columns', None)

statNm         28259
statId         28259
chgerId        28259
chgerType      28259
addr           28259
addrDetail     28259
location       28259
lat            28259
lng            28259
useTime        28055
busiId         28259
bnm            28259
busiNm         28259
busiCall       28259
stat           28259
statUpdDt      28151
lastTsdt       25569
lastTedt       25657
nowTsdt         6028
powerType          0
output         27997
method         27300
zcode          28259
zscode         28259
kind           28239
kindDetail     28239
parkingFree    28258
note            4516
limitYn        28259
limitDetail    21796
delYn          28259
delDetail          3
trafficYn      28259
year           28259
floorNum       28065
floorType      28065
maker          13528
dtype: int64


In [ ]:
#@title 2. 결측치 탐색
# 컬럼별 결측치 개수
# print(df.isnull().sum())
# print(df.isna().sum())

# 결측치 아닌 값 찾기
# print(df.notnull())

# 특정 컬럼의 결측행 조회
print(df[df['useTime'].isnull()])

# 결측치 삭제/대체
# print(df.dropna())
# print(df.fillna(0))

                   statNm    statId chgerId chgerType  \
56     신해운대역(제7508부대 면회실)  ME19D221      01        04   
57     신해운대역(제7508부대 면회실)  ME19D221      91        04   
7362              화명동대림타운  KP000498      01        06   
7363              화명동대림타운  KP000498      02        06   
7388           가아KT-e편한세상  KP001700      01        02   
...                   ...       ...     ...       ...   
11564            센텀동부센트레빌  PI202515      03        02   
11565            센텀동부센트레빌  PI202515      04        02   
11566            센텀동부센트레빌  PI202515      05        02   
11567             주례청구아파트  PI202581      01        02   
11568             주례청구아파트  PI202581      02        02   

                                             addr       addrDetail location  \
56                      부산광역시 해운대구 좌동 사서함 321-16호             null     null   
57                      부산광역시 해운대구 좌동 사서함 321-16호             null     null   
7362                        부산광역시 북구 화명동 금곡대로 268             null     null   

"\n# 데이터 탐색\n# customer_df.shape\n# customer_df.head()\n# customer_df.count()\n# customer_df.info()\n# customer_df.describe(include='all')\n\n# 특정 컬럼들의 데이터 탐색\n# columns_object_dtype = customer_df.columns[customer_df.dtypes=='object']\n# columns_object_dtype\n\n# for col in columns_object_dtype:\n#     # print (sorted(customer_df[col].unique()))\n#     unique_values = sorted(customer_df[col].unique())\n#     print(f'{col}: {len(unique_values)}개')\n#     print(unique_values, '\n')\n"

In [ ]:
#@title 3. 기본 통계 및 분포
# 수치형 기본 통계(카운트, 평균, 표준편차, 최소, 25%, 중앙, 75%, 최대)
print(df.describe())

# 범주형 통계(최빈값 top, 개수 unique, 빈도 freq)
print(df.describe(include='all'))

# 범주형 데이터 빈도
print(df['col'].value_counts())

# 고유값/개수
print(df['col'].unique())
print(df['col'].nunique())

In [ ]:
#@title 4. 데이터 분포 및 이상치
# 수치형 히스토그램
import seaborn as sns

print(df['col'].plot.hist())

# 박스플롯(이상치)
print(df['col'].plot.box())

# 커널 밀도 추정; x값 주변에 데이터가 얼마나 존재하는가(나올 가능성이 얼마나 높은가)를 나타내는 값
print(df['col'].plot.kde())

sns.boxplot(x='col', data=df)
sns.histplot(data=df, x='col')

In [ ]:
#@title 5. 상관 관계/요약/집계
print(df.corr())
sns.heatmap(df.corr())

# 그룹별 집계
print(df.groupby('col').mean())
print(df.groupby('col').size())

# pivot
df.pivot_table(values='매출', index='월', columns='점포', aggfunc='mean')

# 교차표
df.crosstab(df['colA'], df['colB'])

In [ ]:
#@title 6. 필터링/정렬/변환/리포팅
# 조건 필터
print(df.query('col > 100'))

# 정렬
print(df.sort_values(by='col', ascending=False))

# 데이터 변환/새 컬럼 추가
df['col'].map({…})
df.apply()
df.assign()

# 자동 리포트/요약 툴
from ydata_profiling import ProfileReport  # EDA 자동화 라이브러리
profile = ProfileReport(df, title=”Titanic EDA 리포트", explorative=True)  # df에 대한 요약 리포트 객체 생성
profile.to_file('titanic_EDA.html')  # 요약 보고서(결측치, 변수 요약, 상관계수, 그래프 등) 자동 분석 보고서


In [ ]:
######## 0. github 에 commit 하기 위한 환경 설정. access token 등 포함
'''
# https://github.com/magnetclip/artificialneuralnetwork.git
# GIT_TOKEN = "ghp_hox..."
# GIT_USERNAME = "userA"
# GIT_REPO = "DataAnalysisTraning"
# GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPO + ".git"

# from os.path import join
# WORKING_DIR = "DataAnalysisTraining"
# WORKING_TREE = "/content/drive/MyDrive/Colab Notebooks/"+WORKING_DIR
# %cd "{WORKING_TREE}"
#!git init # init을 하면 해당 디렉토리에 init 작업을 진행, 단순히 clone만 하려면, init은 필요 없음
#!git clone "{GIT_PATH}"

# REPO = WORKING_TREE+"/"+GIT_REPO
# %cd "{REPO}"

#!echo "# Data Analysis Traning " >> README.md

# with open("README.md", 'r') as f:
#     c = f.read()
# print(c)

#!git config user.email "userA@mail.com"
#!git config user.name "uA"
#!git branch -a
#!git status
#!git add .
#!git commit -m "commit Data Analysis Training, chapter 1"
#!git commit --amend # 최신 커밋을 수정해서 다시 새로운 커밋 만들기, 이전 최신 커밋은 없어지고 대체됨
#!git push
#!git status
#!git show b602
#!git config alias.history 'log --pretty=oneline'
#!git history
#!git diff 2fdb 8d55
#!git reset --hard  # 잘못해서 hard reset을 한 이후 복구를 하기 위해서라도, push를 적절히 해 두는 것이 중요
'''

######## 1. API를 사용해서 필요한/흥미로운 데이터 수집 (www.data.go.kr)
'''
# 그래프에 한글을 표기하기 위한 설정 (폰트 설치 + ...)
# chatgpt에서 한글을 표기하는 팁; koreanize-matplotlib 검색 >> 다운로드 >> chatgpt에서 라이브러리 설치
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf
# 폰트 설치 후, 런타임 재시작 필요
'''

######## 2. EDA
'''
# 데이터 탐색
# customer_df.shape
# customer_df.head()
pd.set_option('display.max_columns', None)  # colab의 cell 안에서 모든 컬럼을 다 표시하기 위한 설정
# customer_df.count()
# customer_df.info()
# customer_df.describe(include='all')

# 특정 컬럼들의 데이터 탐색
# columns_object_dtype = customer_df.columns[customer_df.dtypes=='object']
# columns_object_dtype

# for col in columns_object_dtype:
#     # print (sorted(customer_df[col].unique()))
#     unique_values = sorted(customer_df[col].unique())
#     print(f'{col}: {len(unique_values)}개')
#     print(unique_values, '\n')
'''

######## 3. 전처리
'''
# 결측값 처리하기 (annual_income 컬럼)
# customer_df.isna().sum()
# customer_df = customer_df.dropna()

# 연령 컬럼
# from datetime import datetime
# customer_df['birth_year'] = datetime.now().year - customer_df['birth_year']
# customer_df['birth_year'] = 2023 - customer_df['birth_year']
# customer_df = customer_df.rename(columns={'birth_year': 'age'})

# 구매 금액 합계 및 구매 횟수 합계 계산하기
# data_amount_total = (
#     customer_df['amount_alcohol']
#     + customer_df['amount_fruit']
#     + customer_df['amount_meat']
#     + customer_df['amount_fish']
#     + customer_df['amount_snack']
#     + customer_df['amount_general']
# )
# customer_df['amount_total'] = data_amount_total
# index_amount_general = customer_df.columns.get_loc('amount_general')
# 특정 컬럼 위치에 삽입
# customer_df.insert(
#     loc=index_amount_general + 1,
#     column='amount_total',
#     value=data_amount_total,
# )

# num_purchase_total = (
#     customer_df['num_purchase_web']
#     + customer_df['num_purchase_store']
#     + customer_df['num_purchase_discount']
# )
# index_num_purchase_discount = customer_df.columns.get_loc('num_purchase_discount')
# customer_df.insert(
#     loc=index_num_purchase_discount + 1,
#     column='num_purchase_total',
#     value=num_purchase_total,
# )

# customer_df.head()

# 불필요한 컬럼 제거하기
# customer_df['revenue'].describe()
# customer_df = customer_df.drop(columns=['ID', 'revenue'])
# customer_df.head()
'''

######## 4. seaborn을 활용한 그래프 가시화
'''
# plt.rc('font', family='NanumGothic')
# RFM (recency, frequency, monetary) 분석
# weight = {}
# weight['recency'] = 0.2
# weight['frequency'] = 0.4
# weight['monetary'] = 0.4

# customer_df['rfm_score'] = (
#     weight['recency'] * customer_df['recency_grade'].astype('int')
#     + weight['frequency'] * customer_df['frequency_grade'].astype('int')
#     + weight['monetary'] * customer_df['monetary_grade'].astype('int')
# )

# def rfm_segment_bins(x):
#     if x < 5/3:
#         return 1
#     elif x <= 7/3:
#         return 2
#     else:
#         return 3

# customer_df['rfm_segment'] = customer_df['rfm_score'].apply(rfm_segment_bins)
# print(customer_df['rfm_segment'].value_counts())
# groupby_rfm_segment = customer_df.groupby('rfm_segment').sum(nu
#         return 2
#     else:
#         return 3

# customer_df['rfm_segment'] = customer_df['rfm_score'].apply(rfm_segment_bins)
# print(customer_df['rfm_segment'].value_counts())
# groupby_rfm_segment = customer_df.groupby('rfm_segment').sum(numeric_only=True).reset_index()
# groupby_rfm_segment['amount_total'].plot(kind='pie', autopct='%.1f%%', labels=[f'{x}등급' for x in grade_labels], title='RFM 고객 세그먼트별 매출 기여도', ylabel='')
